In [25]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pandas as pd
import cv2

import pickle
#import cv2
import math
import time
import h5py
import json
import os
import csv
import random
#import numpy as np
#import pandas as pd
#import matplotlib.image as mpimg
#import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn import preprocessing

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten, Lambda
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

import theano
%matplotlib inline

In [26]:
df = pd.read_csv('driving_log.csv')
center = df[[0]]
center = np.squeeze(center.values)
left = df[[1]]
left = np.squeeze(left.values)
right = df[[2]]
right = np.squeeze(right.values)
steering = df[[3]]
steering = np.squeeze(steering.values)

batch_size = 10

In [27]:
def data_generator(Images,steering,training=0,batch_size=10,threshold=2):
    ctr=0

    while( (training and ctr<len(steering) ) or (not training) ) :
    
        if ((not training) and (ctr>=len(steering))):
            ctr=0

        data,labels=preprocess(Images[ctr:ctr+batch_size],steering[ctr:ctr+batch_size],threshold)    
        ctr +=batch_size

        yield data,labels
        
def crop(img):
    new_size_col=120
    new_size_row=60
    #img=img[60:145,:,:]
    img=cv2.resize(img,(new_size_col,new_size_row), interpolation=cv2.INTER_AREA)
    return img
        
def preprocess(Images,steering,threshold):
    row,col,ch=(mpimg.imread(Images[0][27:35] + '/' + Images[0][36:])).shape

    img = []
    steer = []
        
    for i,path in enumerate(Images):

        I=mpimg.imread(path[27:35] + '/'+ path[36:])

        #if(np.random.randint(2)):
        #    I=augment_brightness_camera_images(I)

            
        img.append(crop(I))
        steer.append(steering[i])

        #if steering[i]<-threshold or steering[i]>threshold :
        #    I=np.fliplr(I)
        #    img.append(crop(I))
        #    steer.append(-steering[i])

    return np.asarray(img),np.asarray(steer)

In [28]:
X_train, X_test, y_train, y_test = train_test_split(center, steering, test_size = 0.2, random_state = 0)
train_data = data_generator(X_train, y_train)
batch_x, batch_y = next(train_data)
print(batch_y)
batch_x, batch_y = next(train_data)
print(batch_y)

[-0.3222892 -0.2801272  0.         0.         0.         0.         0.         0.
  0.         0.       ]
[-0.400292  -0.3177552  0.         0.         0.         0.        -0.2004898
  0.         0.        -0.2859993]


In [29]:
"""
#reading in an image
log = pd.read_csv('driving_log_Arun.csv')
X = []
print(len(log))
for n in range(len(log)):
    img = mpimg.imread(log.ix[n][0][:])#35] + '/' + log.ix[n][0][36:])
    img = cv2.resize(img,(120,60))
    #left = mpimg.imread(log.ix[n][1][28:36] + '/' + log.ix[n][1][37:])
    #right = mpimg.imread(log.ix[n][2][28:36] + '/' + log.ix[n][2][37:])
    #new = np.concatenate((np.concatenate((left,img), axis = 2), right), axis = 2)
    img = img/255 - 0.5
    X.append(img)
X = np.asarray(X)
plt.imshow(X[1])
print(X.shape)
Y = np.asarray(log.ix[:,3])
X_train, X_valid, y_train, y_valid = train_test_split(X, Y, test_size = 0.2, random_state = 0)
print(X_train.shape)
print(X_valid.shape)
#print(Y_val.shape)

X_train,y_train = shuffle(X_train, y_train)
"""

"\n#reading in an image\nlog = pd.read_csv('driving_log_Arun.csv')\nX = []\nprint(len(log))\nfor n in range(len(log)):\n    img = mpimg.imread(log.ix[n][0][:])#35] + '/' + log.ix[n][0][36:])\n    img = cv2.resize(img,(120,60))\n    #left = mpimg.imread(log.ix[n][1][28:36] + '/' + log.ix[n][1][37:])\n    #right = mpimg.imread(log.ix[n][2][28:36] + '/' + log.ix[n][2][37:])\n    #new = np.concatenate((np.concatenate((left,img), axis = 2), right), axis = 2)\n    img = img/255 - 0.5\n    X.append(img)\nX = np.asarray(X)\nplt.imshow(X[1])\nprint(X.shape)\nY = np.asarray(log.ix[:,3])\nX_train, X_valid, y_train, y_valid = train_test_split(X, Y, test_size = 0.2, random_state = 0)\nprint(X_train.shape)\nprint(X_valid.shape)\n#print(Y_val.shape)\n\nX_train,y_train = shuffle(X_train, y_train)\n"

In [44]:
from keras.models import Sequential
from keras.layers import Dense, Input, Activation, Convolution2D, Flatten, MaxPooling2D, Dropout

model = Sequential()
model.add(Convolution2D(128, 3, 3, border_mode = 'valid', input_shape = (60, 120, 3)))
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Convolution2D(128, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(1))
#model.add(Activation('softmax'))

In [45]:
model.summary()

'''
model.compile(optimizer='rmsprop',
          loss='binary_crossentropy',
          metrics=['accuracy'])
'''

model.compile(loss='mse',
              optimizer=Adam(lr = 0.001),
              metrics=['mean_squared_error'])
"""
history = model.fit(data_generator(X_train,y_train,training=0,batch_size=128),
                    samples_per_epoch=len(y_train),
                    validation_data=data_generator(X_test,y_test,training=0,batch_size=128),
                    nb_val_samples=len(y_test), nb_epoch=10, verbose=1)
"""
model.fit_generator(data_generator(X_train,y_train,training=0,batch_size=64),
                    samples_per_epoch=len(y_train),
                    validation_data=data_generator(X_test,y_test,training=0,batch_size=64),
                    nb_val_samples=len(y_test),                    
                    nb_epoch=15,
                    verbose=2)


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_37 (Convolution2D) (None, 58, 118, 128)  3584        convolution2d_input_13[0][0]     
____________________________________________________________________________________________________
activation_49 (Activation)       (None, 58, 118, 128)  0           convolution2d_37[0][0]           
____________________________________________________________________________________________________
maxpooling2d_37 (MaxPooling2D)   (None, 29, 59, 128)   0           activation_49[0][0]              
____________________________________________________________________________________________________
convolution2d_38 (Convolution2D) (None, 27, 57, 32)    36896       maxpooling2d_37[0][0]            
___________________________________________________________________________________________

In [46]:
model.save('model.h5')